In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
MODEL = "llama3.2"

In [ ]:
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
web = Website("https://silviayomdesign.com/")
print(web.title)
print(web.text)

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a very artistic graphic designer's website titled name {website.title}"
    user_prompt += "\nHer creativity of her works are as follow;\
please provide a short summary of her works in markdown. \n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(web))

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_for(web)}
]

In [ ]:
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()["message"]["content"])

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(web)}
    ]

In [ ]:
messages_for(web)

In [ ]:
import ollama
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL, 
        messages=messages
    )
    return response["message"]["content"]

In [ ]:
summarize("https://silviayomdesign.com/")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://silviayomdesign.com/")